In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, date

## Group Assignment
### Team Number: 11
### Team Member Names: Jeffrey, Ray, Baldeep
### Team Strategy Chosen: RISKY (RISKY OR SAFE)

# Initialization of Variables

In [2]:
# List of Tickers to Choose From
ticker_list = pd.read_csv("Tickers.csv")

# Initial Capital (CAD)
capital = 750000

# Trading Fee
trading_fee = 4.95

# Restrictions
avg_month_vol = 150000
min_month_days = 18
currency = ['USD', 'CAD']

# Number of stocks in our Portfolio
num_stocks = 10

# Minimum weight for each stock in the portfolio (in decimals)
min_weight = 1 / (2 * num_stocks)

# Maximum weight for each stock in the portfolio (in decimals)
max_weight = 0.2

# Day that we will purchase stocks for our portfolio
purchase_date = '2023-11-25'

In [3]:
# Portfolios
Portfolio_Final = pd.DataFrame()
Stocks_Final = pd.DataFrame()

# 2. Filtering the tickers

# Before creating our portfolio of stocks, we must first create a filter function to filter out unwanted tickers for this assignment. 

We have to consider:
1) Are the stocks delisted?
2) Are the stocks denominated in USD or CAD?
3) Do the months have at least 18 trading days?
4) Do the stocks have an average monthly volume of at least 150 000 shares? 


1) To do this, we loop through every ticker in the ticker_lst and check if calling fast_info produces an error. If it produces an error, we know the stock is delisted, so we don't add that ticker to the new ticker list. 

2) We get the fast_info of the stock and check if the currency is either USD or CAD. If it is, we add it to the new ticker list. 

3) We use a stock index from the TSX and another from the NYSE and concat the historical data together such that only the days that appear in both are kept. Then, we loop through every month and count the total number of days. If the number of days is less than 18, then we add it to a list and remove the month from the dataframes later. 

4) Using the historical data, we calculate the average monthly volume for each stock by looping through all the valid stocks and then using the mean function. If the average monthly volume is less than 150 000, then we remove the stock from the stock list. 

In [5]:
#define start date and end date variables
start_date = '2023-01-01'
end_date = '2023-10-31'

#define holder variables
stock_hist_data = []
invalid_months = []

#produce a list of all the tickers
ticker_lst = ticker_list[ticker_list.columns[0]]

#filter stocks variable 
def filter_stocks(ticker_lst):
    new_ticker_lst=[]
    
    #loop through every ticker in the ticker_lst and check if it is listed and has a currency of USD or CAD
    for cur_ticker in ticker_lst:
        info = yf.Ticker(cur_ticker).fast_info
        try:
            #check if the currency is USD or CAD
            if(info.currency == 'USD' or info.currency == 'CAD'):
                new_ticker_lst.append(cur_ticker)
        except:
            #output a statement if the stock is delisted
            print(f"{cur_ticker} is not a valid ticker")
    
    #get the historical data of each ticker and add it to a list
    for cur_ticker in new_ticker_lst:
        temp_ticker = yf.Ticker(cur_ticker)
        hist = temp_ticker.history(start=start_date, end=end_date, interval = '1mo')
        stock_hist_data.append(hist)
    
    cad_index = '^GSPTSE' #S&P/TSX Composite Index (CAD)
    usa_index = '^GSPC' #S&P500 Composite Index (USD)
    
    #loop through every month and check if the month has less than 18 trading days
    for cur_month in range(1, 11):
        #get the historical dataframes of the two indexes
        cad_days = yf.Ticker(cad_index).history(start = str(date(2023, cur_month, 1)), end = str(date(2023, cur_month+1, 1)))
        usd_days = yf.Ticker(usa_index).history(start = str(date(2023, cur_month, 1)), end = str(date(2023, cur_month+1, 1)))
        #merge the two dataframes and only keep the duplicates
        total_days = cad_days.reindex(usd_days.index)
        num_days = len(total_days)
        if num_days < min_month_days:
            invalid_months.append(cur_month)
            
    #remove the months that have less than 18 trading days
    for i in range(len(stock_hist_data)):
        cur_hist = stock_hist_data[i]
        for month in invalid_months:
            if month < 10:
                cur_hist.filter(like!=f'2023-0{month}', axis=0)
            else:
                cur_hist.filter(like != f'2023-{month}', axis=0)
    #Calculate the average monthly volume for each stock and remove the stock if it is less than 150 000
    for i in range(len(new_ticker_lst)): 
        cur_ticker = new_ticker_lst[i]
        cur_hist = stock_hist_data[i]
        avgMonthlyVolume = cur_hist['Volume'].mean()
        #check if the average monthly volume is less than 150 000 and remove the stock if it is
        if avgMonthlyVolume < avg_month_vol:
            new_ticker_lst.remove(cur_ticker)
            stock_hist_data.remove(cur_hist)
    return new_ticker_lst

#call the function
filtered_ticker_lst = filter_stocks(ticker_lst)
filtered_ticker_lst

AGN is not a valid ticker
CELG is not a valid ticker
MON is not a valid ticker
RTN is not a valid ticker


['ABBV',
 'ABT',
 'ACN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

# Computing Historical Data

In [13]:
#define start date and end date
compute_start_date = '2022-01-01'
compute_end_date = '2023-11-21'


weekly_pct_change = pd.DataFrame()

#loop through each ticker and add the percent change of the weekly closing price to the dataframe
for ticker in filtered_ticker_lst:
    hist = yf.Ticker(ticker).history(start=compute_start_date, end=compute_end_date, interval = '1wk')
    weekly_pct_change[ticker] = hist.Close.pct_change()

#drop the NaN values
weekly_pct_change = weekly_pct_change.dropna()

#output the weekly percent changes of each stock
weekly_pct_change
    


,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,BK,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-08 00:00:00-05:00,0.007340,-0.054588,-0.046905,0.019262,-0.002559,-0.031869,0.048538,-0.025824,0.028805,0.020375,...,0.045896,0.039083,-0.044331,0.005395,0.032304,0.042187,0.022002,-0.033401,-0.053264,0.036885
2022-01-15 00:00:00-05:00,-0.018551,-0.014717,-0.047713,-0.074140,-0.120237,-0.057363,-0.090813,-0.062409,-0.078479,-0.100943,...,-0.125921,-0.017924,-0.196567,-0.014085,-0.021081,-0.060799,-0.016045,0.000244,-0.022223,-0.128854
2022-01-22 00:00:00-05:00,0.045007,-0.003815,0.021512,-0.006281,0.009359,0.115409,-0.072381,0.021149,0.021268,0.022735,...,0.011399,-0.010163,0.003224,0.016667,-0.000897,0.009394,0.010603,-0.001624,-0.018321,0.054084
2022-01-29 00:00:00-05:00,0.019794,0.034783,0.013563,0.070752,0.094886,0.049644,0.083329,0.052540,-0.016340,0.063135,...,0.075895,0.034432,0.003824,0.031114,0.053987,-0.026707,0.036712,-0.014394,0.133871,0.027893
2022-02-05 00:00:00-05:00,0.009670,-0.032534,-0.052719,-0.004099,-0.027569,0.032069,0.028336,-0.007457,-0.031328,0.008413,...,-0.082632,-0.004222,-0.025656,0.024335,0.020735,-0.040952,-0.010431,-0.014769,-0.050269,-0.019263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-21 00:00:00-04:00,-0.049921,-0.040608,-0.023434,0.000168,0.020532,-0.001837,-0.001944,-0.043330,-0.089066,-0.000720,...,-0.019976,-0.022881,-0.082026,-0.004973,-0.032397,-0.031730,-0.004497,-0.045519,-0.112727,-0.009376
2023-10-28 00:00:00-04:00,0.017923,0.032202,0.080851,0.070721,0.085017,0.081028,0.085481,0.129122,0.063875,0.097976,...,0.122675,0.083128,0.306294,0.101772,0.063288,0.049679,0.011894,0.051606,0.059556,0.158290
2023-11-04 00:00:00-04:00,-0.020011,-0.019512,0.019777,0.001255,0.035786,0.010605,0.008203,-0.026038,-0.099038,0.009061,...,0.039240,0.006290,0.007257,-0.018969,0.012102,-0.011255,0.019420,-0.003017,-0.033249,-0.036348


## Standard Deviation


In [44]:
# Defining 
a = pd.DataFrame()
start_date = '2023-01-01'
end_date = '2023-11-20'


for x in filtered_ticker_lst:
    ticker = yf.Ticker(x)
    hist = ticker.history(start=start_date, end=end_date)
    
    a[(str(x))] = hist['Close']
        
a = a.pct_change()
a = a.drop(a.index[0])

# Function to return the tickers of the firms with the ten highest standard deviations in the original ticker list
def highest_std(dataframe):
    
    # Setting variables
    temp_std = 0 
    list_of_stds = []
    list_of_ten_tickers = []
    
    # Looping through every column in the datafarme
    for x in dataframe.columns:
        temp_std = abs(dataframe[x].std())
        list_of_stds.append(temp_std)
    
    # Return a dataframe with the list of standard deviations
    global stds_dataframe
    stds_dataframe = pd.DataFrame(list_of_stds, index=dataframe.columns, columns=['std'])
    stds_dataframe.sort_values("std", inplace=True, ascending=False)
    
    for x in range (10):
        temp_ticker = (stds_dataframe.index[x])
        list_of_ten_tickers.append(temp_ticker)
        
    
    return list_of_ten_tickers

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.